In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# 데이터 불러 오기
species_list =['"setosa"','"versicolor"','"virginica"']

xy = np.loadtxt('iris.csv',delimiter=',',dtype=np.str,skiprows=1)
xy.shape

x_train = np.float32(xy[:35,1:-1])
y_train = xy[:35,[-1]]
x_test = np.float32(xy[35:50,1:-1])
y_test = xy[35:50,[-1]]

x_train = np.append(x_train , np.float32(xy[50:85,1:-1]),axis= 0)
x_train = np.append(x_train , np.float32(xy[100:135,1:-1]),axis= 0)
y_train = np.append(y_train , xy[50:85,[-1]],axis= 0)
y_train = np.append(y_train , xy[100:135,[-1]],axis= 0)
x_train.shape
y_train.shape

x_test = np.append(x_test, np.float32(xy[85:100,1:-1]),axis=0)
x_test = np.append(x_test, np.float32(xy[135:150,1:-1]),axis=0)
y_test = np.append(y_test, xy[85:100,[-1]],axis=0)
y_test = np.append(y_test, xy[135:150,[-1]],axis=0)
x_test.shape
y_test.shape

C:\Users\kt826\AppData\Local\Temp\ipykernel_31976\2618972588.py:4: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xy = np.loadtxt('iris.csv',delimiter=',',dtype=np.str,skiprows=1)


(45, 1)

In [3]:
# 슬라이싱 완료 후
for i in range(105):
    y_train[i,-1] = np.int32(species_list.index(y_train[i,-1]))
print(y_train)

for i in range(45):
    y_test[i,-1] = np.int32(species_list.index(y_test[i,-1]))
print(y_test)

[['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']]
[['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['0']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['1']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2']
 ['2'

In [4]:
nb_classes = 3 

Y_one_hot = tf.one_hot(y_train,nb_classes)  
print(Y_one_hot.shape)                     
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                    

(105, 1, 3)
(105, 3)


In [5]:
# 변수 초기화 : weight, bias
# (m,n) * (n,l) = (m,l)   : 행렬의 내적 곱셈 공식
# (105,4) * (4,1) = (105,1)
W = tf.Variable(tf.random.normal([4,nb_classes]),name='weight')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')
print(W)
print(b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.18030666, -0.95028627, -0.03964049],
       [-0.7425406 ,  1.3231523 , -0.61854804],
       [ 0.8540664 , -0.08899953,  2.4488697 ],
       [ 0.762508  ,  1.2659615 ,  0.9801489 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0.22652863, 0.8106553 , 0.7466094 ], dtype=float32)>


In [6]:
# 예측 함수(hypothesis) : H(X) = softmax(X*W + B)
def logits(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return tf.nn.softmax(logits(X))  

In [7]:
#  비용함수 구현 방법 2 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                    labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return cost

In [8]:
# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [9]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    optimizer.minimize(cost_func,var_list=[W,b])
    if step % 1000 == 0:
        print('%04d'%step,'cost:[',cost_func().numpy(),']')
print('***** Learning Finished!!') 

***** Start Learning!!
0000 cost:[ 4.8165736 ]
1000 cost:[ 0.17111711 ]
2000 cost:[ 0.105702765 ]
3000 cost:[ 0.08209269 ]
4000 cost:[ 0.07013071 ]
5000 cost:[ 0.06308728 ]
***** Learning Finished!!


In [10]:
# 정확도 측정(평가)

# y_test값을 one-hot인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)  
print(Y_one_hot.shape)                     
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)

def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

correct_predict = tf.equal(predict(x_test),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict,dtype=tf.float32))
print('Accuracy:',accuracy.numpy())  # Accuracy: 0.8064516

(45, 1, 3)
(45, 3)
Accuracy: 1.0
